In [4]:
from flask import request, Response
from flask import Flask
import pprint
from config import netbox_api

In [5]:
def test():
    # Получаем данные через flask от webhook netbox:
    #event = request.json["event"] # type: ignore
    mng_int_id = request.json['data']['id'] # type: ignore
    #prechange = request.json['snapshots']['prechange'] # type: ignore
    #postchange = request.json['snapshots']['postchange'] # type: ignore
    get_device_interface = netbox_api.dcim.interfaces.get(mng_int_id)
    
    #pprint.pprint(request.json)
    local_context = get_device_interface.device.local_context_data
    config_contexts = netbox_api.extras.config_contexts.all()
    #print(type(config_contexts))
    print([config_contexts])
    # Запрашиваем данные через netbox_api:
    """ get_device_interface = netbox_api.dcim.interfaces.get(mng_int_id)
    local_context = get_device_interface.device.local_context_data
    print(local_context) """


    return Response(status=204)

In [ ]:
# Create a Flask instance
app = Flask(__name__)
"""
Webhook POST
    Name:'Fixed IP into DHCPd'
    Content types: 'IPAM > IP Address'
"""
app.add_url_rule("/api/fixed_ip",
                 methods=["POST"],
                 view_func=test)
"""
Webhook POST
    Name:'Change the cable'
    Content types: 'DCIM > Cable'
"""
app.add_url_rule("/api/cable_change",
                methods=['POST'],
                view_func=test)
"""
Webhook POST
    Name:'Update the interface'
    Content types: 'DCIM > Interfaces'
"""
app.add_url_rule("/api/int_update",
                 methods=['POST'],
                 view_func=test)
    
if __name__ == "__main__": 
    app.run(host='0.0.0.0', port=8080)